In [ ]:
# Iman Wahle
# August 2019
# Bootstrap process to identify DSNs and DSNs for cells 
# in TF-varied drifting-grating experiments in Allen Brain Observatory

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import h5py
import progressbar


import matplotlib.pyplot as plt
%matplotlib inline

from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info

# set paths
# got these from here: https://github.com/AllenInstitute/visual_coding_2p_analysis/blob/master/visual_coding_2p_analysis/core.py
save_path = "/allen/programs/braintv/workgroups/cortexmodels/michaelbu/ObservatoryPlatformPaperAnalysis/event_analysis_files_2018_09_25" 
manifest_file = "/allen/programs/braintv/workgroups/cortexmodels/michaelbu/ObservatoryPlatformPaperAnalysis/platform_boc_2018_09_25/manifest.json"

# retrieve data cache
boc = BrainObservatoryCache(manifest_file=manifest_file)

In [6]:
bfp = "/allen/programs/braintv/workgroups/nc-ophys/Iman/direction_flipping/"

In [7]:
# load session info once
csv = pd.read_csv(bfp + "resources/dgtf_events_all.csv")

In [12]:
sessions = np.load(bfp + "resources/dg456_sessions.npy")

In [26]:
# given a cell_id, returns a (2,2,n_trials) array
# that has cell responses at ((pref_dir, null_dir) x (pref_sftf, null_sftf) x 15 trials)

def get_cell_trials(mean_sweep_events, stim_table, pref_dir, null_dir, pref_tf, null_tf, c):
    zz = mean_sweep_events[(stim_table.orientation==pref_dir)&(stim_table.temporal_frequency==pref_tf)][str(c)].values
    oz = mean_sweep_events[(stim_table.orientation==null_dir)&(stim_table.temporal_frequency==pref_tf)][str(c)].values
    zo = mean_sweep_events[(stim_table.orientation==pref_dir)&(stim_table.temporal_frequency==null_tf)][str(c)].values
    oo = mean_sweep_events[(stim_table.orientation==null_dir)&(stim_table.temporal_frequency==null_tf)][str(c)].values
    n_trials = min([len(zz), len(oz), len(zo), len(oo)])
    cell_trials = np.empty((2,2,n_trials))
    cell_trials[0,0,:] = zz[:n_trials]
    cell_trials[1,0,:] = oz[:n_trials]
    cell_trials[0,1,:] = zo[:n_trials]
    cell_trials[1,1,:] = oo[:n_trials]
    return cell_trials

In [9]:
def get_dsi(tf, response):
    pref_dir = 0
    null_dir = 1
    pref = response[pref_dir, tf]
    null = response[null_dir, tf]
    return ((pref - null) / float(pref + null))

In [10]:
def bootstrap(cell_trials, niter, dsi_thresh):

#     # progress bar across iterations
#     bar = progressbar.ProgressBar(maxval=niter, \
#     widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
#     bar.start()
    
    # repeat sampling 1000 times
    drn_accum = 0
    dsn_accum = 0

    for i in range(niter):
        
        # generate response events stats with new sampling
        response_events_sample = np.empty((2, 2)) # (pref_dir=0, null_dir=1) x (pref_tf=0, null_tf=1)
        response_events_sample[:] = np.nan

        # calculate mean response across new sampling at each dir/tf combination
        for d in range(2): # pref_dir, null_dir
            for t in range(2): # pref_tf, null_tf

                # sample n_trials di x tf trials from mean_sweep_events
                n_trials = cell_trials.shape[2]
                sample_trials_idx = np.random.choice(n_trials, n_trials, replace=True)
                sample_mse = cell_trials[d,t,sample_trials_idx]

                # calculate stats
                response_events_sample[d,t] = np.mean(sample_mse)


        # DSI statistics
        DSI_pref = get_dsi(0, response_events_sample)
        DSI_null = get_dsi(1, response_events_sample)
        DSI_ratio = DSI_null / float(DSI_pref)
        
        # apply criteria
        check_drn = (DSI_ratio<0) & (DSI_pref>dsi_thresh)
        check_dsn = (DSI_pref>dsi_thresh)

        drn_accum += check_drn
        dsn_accum += check_dsn
            
#         bar.update(i)
        
#     bar.finish()

    return drn_accum, dsn_accum

In [33]:
niter = 1000
dsi_thresh = 0.25

# progress bar across iterations
bar = progressbar.ProgressBar(maxval=len(sessions), \
widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
    
    
results = pd.DataFrame(columns=('session_id', 'cell_id', 'n_cells', 'drn_accum', 'dsn_accum', 'is_drn', 'is_dsn'))
unprocessed_sessions = []
for s,session_id in enumerate(sessions):

    try:
        #get stim_table and mean_sweep_events
        data_set = boc.get_ophys_experiment_data(int(session_id))
        cells = data_set.get_cell_specimen_ids()
        stim_table = data_set.get_stimulus_table(stim_info.DRIFTING_GRATINGS)    
        data_file_dg = os.path.join(save_path,'DriftingGratings', 
                                    str(session_id) + "_dg_events_analysis.h5")
        mean_sweep_events = pd.read_hdf(data_file_dg, 'mean_sweep_events')
    except:
        unprocessed_sessions.append(session_id)
        continue
        
    session_results = np.zeros((len(cells),2))
    # bootstrap each cell
    for c,cell in enumerate(cells):

        # collect pref/null conditions
        csv_idx = np.where(csv['cell_specimen_id']==cell)[0][0]
        pref_dir = csv['pref_dir'].iloc[csv_idx]
        null_dir = np.mod(pref_dir+180, 360)
        pref_tf = csv['pref_tf'].iloc[csv_idx]
        null_tf = csv['pref_null_tf'].iloc[csv_idx]
        
        if np.isnan(pref_dir):
            unprocessed_sessions.append(session_id)
            break
            
        cell_trials = get_cell_trials(mean_sweep_events, stim_table, \
                                      pref_dir, null_dir, \
                                      pref_tf, null_tf, c)
        
        drn_accum, dsn_accum = bootstrap(cell_trials, niter, dsi_thresh)
        
        results = results.append({'session_id': session_id, \
                        'cell_id' : cell, \
                        'n_cells' : len(cells), \
                        'drn_accum' : drn_accum, \
                        'dsn_accum' : dsn_accum, \
                        'is_drn' : drn_accum>niter*.95, \
                        'is_dsn' : dsn_accum>niter*.95}, \
                       ignore_index=True)
        
    bar.update(s)
bar.finish()
print "Could not process some sessions: "
print unprocessed_sessions

/Users/imanwahle/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


Could not process some sessions: 
[673171528, 712178483, 671618887, 669861524, 715923832, 683257169, 680150733, 645256361, 672206735, 688678766, 674276329, 676503588, 672211004, 710778377, 670395999, 671164733, 680156911, 673475020, 710504563, 671164927, 675477919, 692345003, 673914981, 670728674, 676024666]


In [73]:
# results.to_csv(bfp + "bootstrap_code/results/bootstrap_results_dsi25.csv")

In [53]:
results.head()

,session_id,cell_id,n_cells,drn_accum,dsn_accum,is_drn,is_dsn
0,556353209,557374817,21,585,986,False,True
1,556353209,557374888,21,22,954,False,True
2,556353209,557374807,21,409,438,False,False
3,556353209,557374802,21,480,616,False,False
4,556353209,557374843,21,628,801,False,False
